In [ ]:
from pathlib import Path
import duckdb

# Compute the database path relative to this notebook using cwd
notebook_dir = Path.cwd()
db_path = (notebook_dir.parent / "database" / "geocoder.duckdb").resolve()
con = duckdb.connect(str(db_path))

print(f"Connected to {db_path}")

In [ ]:
# List all available tables
tables = con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main' ORDER BY table_name").fetchall()
tbl_ls = [table[0] for table in tables]
print(f"Available tables ({len(tables)}):")
for table in tables:
    print(f"  - {table[0]}")
print(tbl_ls)

In [ ]:
with duckdb.connect(str(db_path)) as conn:
    for tbl in tbl_ls:
        df = conn.execute(f"SELECT * FROM {tbl} limit 10").df()
        globals()[f"result_{tbl}"] = df
        print(f"Loaded first 10 rows from {tbl} into variable 'result_{tbl}'")

In [ ]:
result_addr
result_edges
result_featnames

In [ ]:
con.close()